Building PR curve

In [ ]:
import torch

classes = ["plane", "car", "bird"]
labels = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2])
predictions = torch.tensor(
    [
        [900, 0.2, 600],  # True plane
        [40, 22, 36],  # True plane
        [50, 0.4, -20],  # True plane
        [-2, -200, -30],  # True plane
        [4, 0.6, -6],  # True plane
        [25, 22, 23],  # True plane
        [2, 1.8, 1.2],  # True plane
        [34, -45, 28],  # True plane
        [1, 0.9, -8],  # True plane
        [0.01, 0, 0],  # True plane
        [0.09, 0.01, 0],  # True plane
        [12, 10, 10],  # True plane
        [100, 0, 0],  # True plane
        [5, 4, 4],  # True plane
        [70, 15, 90],  # False bird real plane
        [-56, 0, -20],  # True car
        [-7, -2, -12],  # True car
        [12, 90, 100],  # True bird
        [55, -80, 160],  # True bird
    ]
)

probabilities = torch.nn.functional.softmax(predictions, dim=1)
print(probabilities)
# check that all sums equal to one
print(probabilities.sum(dim=1))

# plane prob
plane_prob = probabilities[:, classes.index("plane")]
print(plane_prob)
plane_gt = labels == classes.index("plane")
print(plane_gt)

In [ ]:
gt = probabilities.argmax(dim=1) == labels
result = pd.DataFrame(probabilities.numpy(), columns=classes)
result = pd.concat([result, pd.DataFrame(labels.numpy())], axis=1)
result = pd.concat([result, pd.DataFrame(gt.numpy())], axis=1)
result

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix

values, indexes = predictions.max(dim=1)
print(indexes)
conf_matrix = pd.DataFrame(confusion_matrix(labels, indexes))

conf_matrix.columns = classes
conf_matrix.index = classes

conf_matrix = conf_matrix.rename_axis("Real")
conf_matrix = conf_matrix.rename_axis("Predicted", axis="columns")

conf_matrix

Результаты для одного класса

In [ ]:
import numpy as np

prob = pd.DataFrame(plane_prob, columns=["prob"])
gt = pd.DataFrame(plane_gt, columns=["GT"])
data = pd.concat([prob, gt], axis=1)
pd.options.display.float_format = "{:,.2f}".format
data.head(6)

Сортируем таблицу по вероятности

In [ ]:
data = data.sort_values("prob", ascending=False)
data

Строим кривую

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))
# cnt = len(data)

x = [0]
y = [1]
for index, row in data.iterrows():
    threshold = row["prob"]
    TP = len(data.query(f"prob >= {threshold} and GT"))
    FP = len(data.query(f"prob >= {threshold} and not GT"))
    precision = TP / (TP + FP)

    FN = len(data.query(f"prob < {threshold} and GT"))
    recall = TP / (TP + FN)

    plt.scatter(recall, precision)
    plt.annotate(f"{threshold:.2f}", (recall, precision))
    y.append(precision)
    x.append(recall)

x.append(1)
y.append(0)


plt.plot(x, y)
plt.show()

Построим ту же кривую при помощи sklearn

In [ ]:
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(data["GT"], data["prob"])

print(thresholds)
plt.plot(recall, precision)

plt.show()

https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html#sphx-glr-auto-examples-model-selection-plot-precision-recall-py

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

display = PrecisionRecallDisplay.from_predictions(
    data["GT"], data["prob"], name="Plane"
)

In [ ]:
from sklearn.metrics import f1_score

values, indexes = predictions.max(dim=1)
f1_mean = f1_score(labels, indexes, average="macro")

f1_mean

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
for class_name in classes:
    class_num = classes.index(class_name)
    class_prob = probabilities[:, class_num]
    class_gt = labels == class_num
    writer.add_pr_curve(class_name, class_gt, class_prob)
writer.close()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs